In [1]:
import soundfile as sf
import os
import numpy as np
import librosa
import time

In [2]:
wanted_samples = 30000

In [3]:
def convertToMFCC(data, samplerate):
    mfcc = librosa.feature.mfcc(y = data, sr = samplerate, n_mfcc=40)
    return mfcc

In [4]:
def halfSecondSplit(samples, samplerate):
    """Takes 'Audio Data' samples and 'Samplerate', splits it into 0.5 second fragments"""

    #disregards less than half a second of audio files
    halfSecondSamples = samplerate/2
    samples = samples[:int((len(samples)-(len(samples) % halfSecondSamples)))]
    array = np.reshape(samples, (int(samples.size/halfSecondSamples), int(halfSecondSamples)))

    return array

In [5]:
def loadAudio(path): 
    audiodata, sr = sf.read(path)
    
    if not sr == 44100:
        audiodata = librosa.resample(audiodata, sr, 44100)
        sr = 44100
    
    halfseconds = halfSecondSplit(audiodata,sr)
    return halfseconds, sr

In [6]:
def wavsToData(path):
    global wanted_samples
    timestamp = time.perf_counter()

    samples = []
    
    files = np.array(os.listdir(path))
    np.random.shuffle(files)
    
    for filename in files:
        if filename.endswith('.wav'):
            halfseconds, sr = loadAudio(path + filename)
            for halfSecond in halfseconds:
                samples.append(convertToMFCC(halfSecond, sr))
                
        print('Progress: ' + str((len(samples) / wanted_samples) * 100), end='\r')
        if len(samples) >= wanted_samples:
            break
    duration = time.perf_counter() - timestamp
    print('duration: ' + str(duration) + ' seconds            ')
    
    return samples

In [7]:
def main():
    global wanted_samples
    
    path = 'audio/Split_Labels/'
    
#     subject = 'CLEAN_SPEECH'
    subject = 'NO_SPEECH'
#     subject = 'SPEECH_WITH_MUSIC'
#     subject = 'SPEECH_WITH_NOISE'
    
    data = wavsToData(path + subject + '/')
    datafile = np.array(data)
    
    if subject == 'NO_SPEECH':
        labels = np.repeat([False], len(datafile))
        print('labeled as False')
    else:
        labels = np.repeat([True], len(datafile))
        print('labeled as True')
        
    print(datafile.shape, labels.shape)
    
    np.save(path + 'outputs/' + subject + '_' + str(wanted_samples) + '_samples.npy', datafile)
    np.save(path + 'outputs/' + subject + '_' + str(wanted_samples) + '_labels.npy', labels)

In [8]:
main()

duration: 134.54830333497375 seconds            
labeled as False
(30007, 40, 44) (30007,)
